# DSBs $\to$ Genomic Loci

In [1]:
using Pkg
Pkg.activate("..")
Pkg.status()

  Activating project at `~/Documents/Kaizen/code/space-behaviour`
   Resolving package versions...
  No Changes to `~/Documents/Kaizen/code/space-behaviour/Project.toml`
  No Changes to `~/Documents/Kaizen/code/space-behaviour/Manifest.toml`


Status `~/Documents/Kaizen/code/space-behaviour/Project.toml`
  [46ada45e] Agents v6.2.10
  [336ed68f] CSV v0.10.15
⌃ [13f3f980] CairoMakie v0.13.10
  [a93c6f00] DataFrames v1.7.0
  [0c46a032] DifferentialEquations v7.16.1
  [31c24e10] Distributions v0.25.120
  [587475ba] Flux v0.16.5
  [86223c79] Graphs v1.13.0
⌅ [a98d9a8b] Interpolations v0.15.1
⌅ [033835bb] JLD2 v0.4.54
⌅ [ee78f7c6] Makie v0.22.10
  [c3e4b0f8] Pluto v0.20.13
⌃ [7f904dfe] PlutoUI v0.7.68
  [90137ffa] StaticArrays v1.9.14
⌃ [2913bbd2] StatsBase v0.34.5
⌃ [fce5fe82] Turing v0.39.8
  [9a3f8284] Random v1.11.0
Info Packages marked with ⌃ and ⌅ have new versions available. Those with ⌃ may be upgradable, but those with ⌅ are restricted by compatibility constraints from upgrading. To see why use `status --outdated`


## GCR Simulation

In [2]:
# Import the physics from notebook 02
include("../modules/GCRSimulator/GCRSimulator.jl")
using CSV, DataFrames, Interpolations, Distributions

# Recreation of key data (copy the essential parts)
PARTICLE_PARAMS = Dict(
    :proton => (γ = 2.7, cutoff = 1.0e4, A = 1.0, weight = 0.87),
    :helium => (γ = 2.58, cutoff = 1.0e4, A = 0.1, weight = 0.12),
    :heavy => (γ = 2.6, cutoff = 1.0e4, A = 0.01, weight = 0.01),
)

# Generate eventlist (same as notebook 02)
sim = GCRSimulator(PARTICLE_PARAMS; area=1.0)
times, species, energies, directions = simulate_events(sim, round(Int, sim.λ * 3600 * 24))

println("λ = ", sim.λ, " events/s")
println("First 5 events:")
for i in 1:5
    println((times[i], species[i], energies[i], directions[i]))
end

eventlist = [(time = times[i], particle = species[i], energy = energies[i], direction = directions[i]) for i in 1:length(times) if energies[i] > 10]
println("Selected $(length(eventlist)) events")

λ = 8.264562861287374 events/s
First 5 events:
(0.08045716449487317, :proton, 1.6586120459619966, [-0.7892707797841357, 0.40204891553931715, 0.46412100328750616])
(0.1692884246196395, :proton, 1.6246861583937644, [0.19415822454320555, -0.5828893539915502, 0.7890136784907746])
(0.4917952629540153, :proton, 1.7905287345888437, [0.03972011804885405, 0.6813553391984611, 0.730874280548943])
(0.5292060138108665, :proton, 1.906590133840186, [0.6390268158138432, 0.7650704242802885, -0.07944793617457756])
(0.6484388980478354, :proton, 1.6629954143968988, [0.008907550886827407, 0.2690242495616623, 0.9630922119324727])
Selected 14808 events


## LET LookUp

In [9]:
include("../modules/Utilities/lookupLET.jl")

protonLET = loadLETfromNIST("protons-in-water.tsv")
alphaLET = loadLETfromNIST("alpha-in-water.tsv")
ironLET = loadLETfromSRIM("iron-in-water.tsv")

lookupLET = Dict(
    :proton => protonLET,
    :helium => alphaLET,
    :heavy => ironLET
)

Dict{Symbol, Interpolations.Extrapolation{Float64, 1, Interpolations.GriddedInterpolation{Float64, 1, Vector{Float64}, Gridded{Linear{Throw{OnGrid}}}, Tuple{Vector{Float64}}}, Gridded{Linear{Throw{OnGrid}}}, Flat{Nothing}}} with 3 entries:
  :proton => 131-element extrapolate(interpolate((::Vector{Float64},), ::Vector…
  :helium => 120-element extrapolate(interpolate((::Vector{Float64},), ::Vector…
  :heavy  => 72-element extrapolate(interpolate((::Vector{Float64},), ::Vector{…

## PARTRAC DSB

In [14]:
include("../modules/Utilities/partracDSByield.jl")

partracDSByield (generic function with 1 method)

## Modelling Nuclear Geometry

In [13]:
using Random, LinearAlgebra

nucleus_radius = 5.0

function impact(radius::Float64)
    # Sample impact parameter for isotropic flux (hits only)
    u = rand()
    b = radius * sqrt(u)
    
    # Calculate chord length through sphere
    chord = 2 * sqrt(radius^2 - b^2)
    
    return chord, b
end

# Test it - now we'll see all the results
println("Testing impact function:")
println("Nucleus radius: $nucleus_radius μm")
for i in 1:10
    chord, b = impact(nucleus_radius)
    println("  Impact parameter: $(round(b, digits=2)) μm → Chord length: $(round(chord, digits=2)) μm")
end

# Quick stats
chords = [impact(nucleus_radius)[1] for _ in 1:1000]
println("\nChord length statistics (1000 samples):")
println("  Mean: $(round(mean(chords), digits=2)) μm")
println("  Max (diameter): $(round(maximum(chords), digits=2)) μm")
println("  Expected max: $(round(2*nucleus_radius, digits=2)) μm")

Testing impact function:
Nucleus radius: 5.0 μm
  Impact parameter: 0.97 μm → Chord length: 9.81 μm
  Impact parameter: 4.9 μm → Chord length: 2.01 μm
  Impact parameter: 2.82 μm → Chord length: 8.25 μm
  Impact parameter: 4.31 μm → Chord length: 5.07 μm
  Impact parameter: 3.37 μm → Chord length: 7.4 μm
  Impact parameter: 4.31 μm → Chord length: 5.07 μm
  Impact parameter: 4.72 μm → Chord length: 3.33 μm
  Impact parameter: 2.64 μm → Chord length: 8.49 μm
  Impact parameter: 1.74 μm → Chord length: 9.38 μm
  Impact parameter: 4.7 μm → Chord length: 3.41 μm

Chord length statistics (1000 samples):
  Mean: 6.57 μm
  Max (diameter): 9.99 μm
  Expected max: 10.0 μm


In [ ]:
function mapDSBs(events)

    # Retrive Linear Energy Transfer for the event
    LET = lookupLET[event.particle](event.energy)

    # Calculate
    chord, impact = impact(nuclear_radius)



